In [ ]:
%matplotlib notebook

import math
import numpy as np
import matplotlib.pyplot as plt
import sympy

import geometrik.threed as g3d

from goto.globe.plot import GlobePlotMpl

In [ ]:
u = sympy.symbols('u')

L'idée est de prendre le plan equatorial et de l'incliner d'un angle $\theta$ autour de l'axe $Ox$

In [ ]:
Ex = g3d.Vector(1, 0, 0)
Ey = g3d.Vector(0, 1, 0)
Ez = g3d.Vector(0, 0, 1)

In [ ]:
theta, psi = sympy.symbols('theta psi')

Nz = g3d.Vector(0, -sympy.sin(theta), sympy.cos(theta))
Ny = g3d.Vector(0, sympy.cos(theta), sympy.sin(theta))
Nx = g3d.Vector(1, 0, 0)

m_val = {'theta': math.pi / 4, 'psi': 0.3}

In [ ]:
Ux = g3d.Vector(sympy.cos(psi), sympy.sin(psi), 0)

In [ ]:
Mx = Ux.project(Nz)
Mx.x

In [ ]:
Mx.y.simplify()

In [ ]:
Mx.z.simplify()

In [ ]:
Mz = (Ux * Nz) * Nz
Mz.z

In [ ]:
with GlobePlotMpl() as gpl :
    gpl.add_point(Nx.subs(m_val), 'Nx', 'r')
    gpl.add_point(Ny.subs(m_val), 'Ny', 'g')
    gpl.add_point(Nz.subs(m_val), 'Nz', 'b')
    gpl.add_point(Mz.subs(m_val), 'Mz', 'b')
    gpl.add_great_circle(Nz.subs(m_val), 'k')
    gpl.add_point(Ux.subs(m_val), 'Ux', 'yellow')
    gpl.add_point(Mx.subs(m_val), 'Mx', 'purple')

In [ ]:
Nx.subs({'theta': 0.3})
R = sympy.symbols('R')

In [ ]:
r = Ux.angle_to(Mx).simplify()
r

In [ ]:
r_sol = sympy.solve(r - R, theta)
r_sol

In [ ]:
r_sol[0]

In [ ]:
r_sol[1]

In [ ]:
r_sol[2]

In [ ]:
r_sol[3].simplify()

In [ ]:
theta_lst = [float(r_sol[i].subs({'psi': 0.3, 'R': 0.1})) for i in range(4)]
theta_lst

In [ ]:
C = g3d.Vector.compose(Ex, Ey, 0.3)

Pp = g3d.Vector(0, -sympy.sin(theta), sympy.cos(theta)).subs({'theta': theta_lst[2]})
Pn = g3d.Vector(0, -sympy.sin(theta), sympy.cos(theta)).subs({'theta': theta_lst[3]})
Mp = C.project(Pp).normalized()
Mn = C.project(Pn).normalized()

with GlobePlotMpl() as gpl :
    gpl.add_point(Ex, 'Ex', 'r')
    gpl.add_point(Ey, 'Ey', 'g')
    gpl.add_point(Ez, 'Ez', 'b')
    gpl.add_point(Mp, 'Mp', 'cyan')
    gpl.add_point(Mn, 'Mn', 'cyan')

    gpl.add_point(g3d.Vector(sympy.cos(psi), sympy.sin(psi), 0).subs({'psi': 0.3}), 'C', 'orange')
    gpl.add_circle(C, 0.1, 'orange')    
    gpl.add_great_circle(Pp, 'magenta')
    gpl.add_great_circle(Pn, 'magenta')


In [ ]:
r_lst = list()
t_lst = np.linspace(0.0, math.pi/2, 100)
u_lst = list()
k = math.pi / 3
for t in t_lst :
    r_lst.append(math.acos(math.sqrt(1 - (math.sin(k)**2 * math.sin(t)**2))))
    u_lst.append(math.sin(t)* k)

plt.figure()
plt.plot(t_lst, r_lst)
plt.plot(t_lst, u_lst)
plt.show()

La question se pose maintenant de manière plus générale : étant donné deux points A et B, et un rayon de virage donné, trouver les 4 segments qui forment les tangentes à ces cercles.

In [ ]:
Ax, Ay, Az = sympy.symbols('Ax Ay Az')
A = g3d.Vector(Ax, Ay, Az)

In [ ]:
Bx, By, Bz = sympy.symbols('Bx By Bz')
B = g3d.Vector(Bx, By, Bz)

Cx est le point exactement au milieu des deux cercles, comme ils sont de taille égales, c'est également le point de croisement des tangentes. 

In [ ]:
Cx = (A + B).normalized()
Cx.x

Cz est le vecteur perpendiculaire au plan equatorial contenant A et B, et Cy construit la base Cx, Cy, Cz, directe

In [ ]:
Cz = (A @ B).normalized()
Cz.x

In [ ]:
Cy = (Cz @ Cx)

In [ ]:
Psi_intern = Cx.angle_to(B)